<a href="https://colab.research.google.com/github/zee404-code/DartsTimeSeries/blob/main/darts06(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install darts

In [ ]:
#from darts.datasets import HeartRateDataset      -> is stationary
#data = HeartRateDataset().load()
from darts.datasets import AirPassengersDataset     # -> not stationary
data = AirPassengersDataset().load()
data.plot()

In [4]:
from darts.utils.statistics import check_seasonality, extract_trend_and_seasonality,remove_seasonality, remove_trend, stationarity_test_adf, stationarity_test_kpss, stationarity_tests
from darts.utils.model_selection import train_test_split
from darts.utils.utils import ModelMode, SeasonalityMode
from darts.models import AutoARIMA, ARIMA, VARIMA

In [ ]:
check1 = stationarity_tests(data, 0.05, 0.05)
print(check1)
#stationary is based on data. Depending on the stationarity, forecasting models are used
#remove non-stationary attributes, apply ARIMA
#if seasonality exists, use SARIMA (seasonal ARIMA)
#VARIMA is used for multivariate data
#AutoARIMA is for optimal p,d,q values

In [6]:
train, test = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)

In [ ]:
train.plot(label = 'train')
test.plot(label = 'test')

In [ ]:
model = ARIMA()
model.fit(train)
pred = model.predict(len(test))
train.plot(label = 'train')
test.plot(label = 'test')
pred.plot(label = 'pred')

In [ ]:
model2 = AutoARIMA()
model2.fit(train)
pred = model2.predict(len(test))
train.plot(label = 'train')
test.plot(label = 'test')
pred.plot(label = 'pred')

In [10]:
data2, data3 = extract_trend_and_seasonality(data, freq=12, model=ModelMode.MULTIPLICATIVE, method='naive')
#data 2 is trend
#data 3 is seasonality

In [ ]:
check2 = stationarity_tests(data2, 0.05, 0.05)
print(check2)
#if data 2 is false, means trend exists

In [62]:
check3 = stationarity_tests(data3, 0.05, 0.05)
print(check3)
#(if data 3 is true, means seasonality does not exist)
#the above comment isn't working out. stationarity tests is still giving an error.
da4 = remove_seasonality(data, freq=None, model=SeasonalityMode.MULTIPLICATIVE, method='naive')
for m in range(2, 25):
    is_seasonal, period = check_seasonality(da4, m=m, alpha=0.05)
    print('running {}'.format(m))
    if is_seasonal:
        print("There is seasonality of order {}.".format(period))

True
running 2
running 3
running 4
running 5
running 6
running 7
running 8
running 9
running 10
running 11
running 12
running 13
running 14
running 15
running 16
running 17
running 18
running 19
running 20
running 21
running 22
running 23
running 24


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/stattools.py:2022: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


In [ ]:
#visualizations
data2.plot() #shows trend exists

In [ ]:
data3.plot() #shows lack of seasonality

In [15]:
#now that i know seasonality doesnt exist, i can simply remove trend only
data4 = remove_trend(data, model=ModelMode.MULTIPLICATIVE, method='naive')
#this trend is being removed from the original data

In [ ]:
data4.plot()

In [ ]:
#check for seasonality in data4
check3 = stationarity_tests(data4, 0.05, 0.05)
print(check3)

In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#since, check3 is true. data is stationary now. use ARIMA()
train2, test2 = train_test_split(data4, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
model3 = ARIMA() # AutoARIMA isnt working
model3.fit(train2)
pred2 = model3.predict(len(test2))
train2.plot(label = 'train')
test2.plot(label = 'test')
pred2.plot(label = 'pred')

In [ ]:
data.is_univariate    #if this is false, use VARIMA

In [23]:
def vis(train3, test3, pred3):
  train3.plot(label = 'train')
  test3.plot(label = 'test')
  pred3.plot(label = 'pred')

In [ ]:
if stationarity_tests(data, 0.05, 0.05): 
  print('this runs 1')
  if data.is_univariate:
    print('this runs 2')
    train, test = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
    model = ARIMA()
    model.fit(train)
    pred = model.predict(len(test))
    vis(train, test, pred)
  else:
    print('this runs 3')
    train, test = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
    model = VARIMA()
    model.fit(train)
    pred = model.predict(len(test))
    vis(train, test, pred)
else:
  print('this runs 4')
  data2, data3 = extract_trend_and_seasonality(data, freq=12, model=ModelMode.MULTIPLICATIVE, method='naive')
  #either data has trend, seasonality or both
  if (not stationarity_tests(data2, 0.05, 0.05)) and (not stationarity_tests(data3, 0.05, 0.05)) :
    print('this runs 5')
    #means trend and stationary both exists, remove both
    #data4 = remove_trend(data, model=ModelMode.MULTIPLICATIVE, method='naive').remove_seasonality(data, freq=None, model=SeasonalityMode.MULTIPLICATIVE, method='naive')
    data4 = remove_seasonality(data, freq=None, model=SeasonalityMode.ADDITIVE, method='STL')
    # method = 'naive' with model = 'multiplicative' if only seasonal
    # method = 'STL' with model = 'additive' if both seasonal and trend
  elif not stationarity_tests(data2, 0.05, 0.05):
    print('this runs 6')
    #means only trend exists, remove it
    data4 = remove_trend(data, model=ModelMode.MULTIPLICATIVE, method='naive')
  else:
    print('this runs 7')
    #means only seasonality exists, remove it
    data4 = remove_seasonality(data, freq=None, model=SeasonalityMode.MULTIPLICATIVE, method='naive')

#check stationarity again
print('this runs 8')
check = stationarity_tests(data4, 0.05, 0.05)
print(check)

 #if true use ARIMA, if false (check if seasonality still exists, if true use SARIMA)

if not stationarity_tests(data4, 0.05, 0.05):
  if check_seasonality(data4, m=None, max_lag=24, alpha=0.05):
      print('use SARIMA')

print('this runs 9')
train2, test2 = train_test_split(data4, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)

if data.is_univariate:
  print('this runs 10')
  model2 = ARIMA()
else:
  print('this runs 11')
  model2 = VARIMA()

print('this runs 12')
model2.fit(train2)
pred2 = model2.predict(len(test2))
vis(train2, test2, pred2)


In [ ]:
######## box-cox, differencing and pipelines #################
# box-cox removes both seasonality and trend. It works on 
# box-cox -> data show variation that increases or decreases with the level of the series. log and power transformations
# lambda = 0 -> log transformations are used
# lambda not equal to 0, power transformations are used followed by simple scaling.
# differencing only removes seasonality. Isn't removing trend.
for m in range(2, 25):
    is_seasonal, period = check_seasonality(data, m=m, alpha=0.05)
    if is_seasonal:
        print("There is seasonality of order {}.".format(period))

In [ ]:
dat = data.diff() #simple differencing isnt removing seasonality
for m in range(2, 25):
    is_seasonal, period = check_seasonality(dat, m=m, alpha=0.05)
    print('running {}'.format(m))
    if is_seasonal:
        print("There is seasonality of order {}.".format(period))

In [126]:
dat = data.diff(periods = 12) #12 given that the seasonality is of order 12
# n = 2 is creating seasonality of order 23. n is differencing steps
for m in range(2, 25):
    is_seasonal, period = check_seasonality(dat, m=m, alpha=0.05)
    print('running {}'.format(m))
    if is_seasonal:
        print("There is seasonality of order {}.".format(period))

#seasonality removed through differencing. check if dat is stationary.
c = stationarity_tests(dat, 0.05, 0.05)
print(c) #it isnt. check for trend.

dat3 = remove_trend(dat, model=ModelMode.ADDITIVE, method='naive')
c2 = stationarity_tests(dat3, 0.05, 0.05)
print(c2) #it is. both trend and seasonality have been removed. 

running 2
running 3
running 4
running 5
running 6
running 7
running 8
running 9
running 10
running 11
running 12
running 13
running 14
running 15
running 16
running 17
running 18
running 19
running 20
running 21
running 22
running 23
running 24
False
True


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/stattools.py:2022: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


In [115]:
##### box-cox
from darts.dataprocessing.transformers import BoxCox
transformer = BoxCox(name='BoxCox',lmbda = None)
series_transformed = transformer.fit_transform(data)
print(series_transformed.head())
c3 = stationarity_tests(series_transformed, 0.05, 0.05)
print(c3)
for m in range(2, 25):
    is_seasonal, period = check_seasonality(series_transformed, m=m, alpha=0.05)
    print('running {}'.format(m))
    if is_seasonal:
        print("There is seasonality of order {}.".format(period))
dat4 = remove_trend(series_transformed, model=ModelMode.MULTIPLICATIVE, method='naive')
c4 = stationarity_tests(series_transformed, 0.05, 0.05)
print(c4)

<TimeSeries (DataArray) (Month: 5, component: 1, sample: 1)>
array([[[6.82749005]],

       [[6.93282224]],

       [[7.16189151]],

       [[7.11461078]],

       [[6.98378687]]])
Coordinates:
  * Month      (Month) datetime64[ns] 1949-01-01 1949-02-01 ... 1949-05-01
  * component  (component) object '#Passengers'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
False
running 2
running 3
running 4
running 5
running 6
running 7
running 8
running 9
running 10
running 11
running 12
There is seasonality of order 12.
running 13
running 14
running 15
running 16
running 17
running 18
running 19
running 20
running 21
running 22
running 23
running 24
False


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/stattools.py:2018: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/stattools.py:2018: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


In [91]:
#from darts.dataprocessing.transformers import diff, Diff

In [ ]:
#final verdict is that use stationary tests to detect trends. use check_seasonality to detect seasonality.
#use remove trends. use either remove seasonality or used diff(periods = order of seasonality)

from darts.dataprocessing.pipeline import Pipeline
pipeline = Pipeline([BoxCox(name='BoxCox', lmbda=None, optim_method='mle', n_jobs=1, verbose=True)],copy=False, verbose=True, n_jobs=None)
# (1) remove trend, (2) remove seasonality
# (3) check stationarity and seasonality
series_transformed = pipeline.fit_transform(data)
c4 = stationarity_tests(series_transformed, 0.05, 0.05)
print(c4)
for m in range(2, 25):
    is_seasonal, period = check_seasonality(series_transformed, m=m, alpha=0.05)
    print('running {}'.format(m))
    if is_seasonal:
        print("There is seasonality of order {}.".format(period))

#box-cox isnt removing either trend or seasonality

In [ ]:
#from darts.dataprocessing.transformers import Mapper
#transform = Mapper(diff(periods = 12))
#series_transformed = transformer.transform(data)

In [94]:
data.has_static_covariates

False

In [ ]:
#from darts.dataprocessing.transformers import (
#    Scaler,
#    MissingValuesFiller,
#    Mapper,
#    InvertibleMapper,
#    boxcox,
#    diff
#)
#from darts.dataprocessing import Pipeline

#scaler = Scaler()
#rescaled = scaler.fit_transform(data)

#dif = Mapper(
#    TimeSeries.diff(periods = 12)
#)

#pipeline = Pipeline([dif, BoxCox(name='BoxCox', lmbda=None, optim_method='mle', n_jobs=1, verbose=True)])
#transformed = pipeline.fit_transform(data)
#transformed.plot()